In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
!ls

base  data_repository  notebooks  requirements.txt  src


In [23]:
cd /content/drive/MyDrive/Academic/Topics/AI/Machine\ Learning\ Dr.\ Montazeri/Project/ml_mda

/content/drive/MyDrive/Academic/Topics/AI/Machine Learning Dr. Montazeri/Project/ml_mda


In [24]:
!pip install torch_geometric

In [25]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html


# Requirements

In [26]:
import torch

from torch.optim import Adam
from torch_geometric.nn import ComplEx, DistMult, RotatE, TransE
from torch_geometric.data import Data

from base import OptimizerConfig, cross_validation
from base import SimplePytorchData, SimplePytorchDataTrainTestSplit
from base import SimpleTrainer, SimpleTester
from src.config import SimpleClassifierConfig, GraphAutoEncoderConfig, KGEConfig
from src.features import get_relations, get_entities, get_associations, get_homogeneous_graph, get_kge_pair_embedd_for_training_data
from src.models import SimpleMDAClassifier, SimpleMDAClassifierFactory
from src.utils import train_test_sampler, prj_logger
from torch_geometric.nn import GCNConv

In [27]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [28]:
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(stream=sys.stdout)
    ],
    force=True
)

# DistMult

## Config

In [29]:
kge_optimizer_config = OptimizerConfig()
kge_optimizer_config.optimizer = torch.optim.Adam
kge_optimizer_config.lr = 0.01
kge_optimizer_config.batch_size = 1000
kge_optimizer_config.n_epoch = 40
kge_optimizer_config.exp_name = "Optimizer for Graph Auto Encoder"
kge_optimizer_config.device = device
kge_optimizer_config.report_size = device

In [30]:
kge_model_config = KGEConfig()
kge_model_config.kge = DistMult
kge_model_config.hidden_channels = 32

## Embedding

In [31]:
md_embed = get_kge_pair_embedd_for_training_data(kge_model_config, kge_optimizer_config)

2024-02-12 09:42:38,284 [INFO] Calling get_node2vec_pair_embedd on cuda device ...
2024-02-12 09:42:38,288 [INFO] Calling get_homogeneous_graph
2024-02-12 09:42:39,588 [INFO] homogeneous data : Data(x=[66911, 1], edge_index=[2, 633662])
2024-02-12 09:42:39,593 [INFO] Calling get_kge_embedd on cuda device ...
2024-02-12 09:42:39,594 [INFO] Calling get_knowledge_graph
2024-02-12 09:42:40,518 [INFO] knowledge graph data : Data(num_nodes=66911, edge_index=[2, 633662], edge_type=[633662])
2024-02-12 09:42:40,519 [INFO] Setting num relations and num nodes for kge config to 39 and 66911
2024-02-12 09:42:40,522 [INFO] Creating KGE model ...
2024-02-12 09:42:40,525 [INFO] Initialing MDATransE with model_config {'model_name': None}
2024-02-12 09:42:40,559 [INFO] Training KGE ...
2024-02-12 09:42:40,559 [INFO] Running KGETrainer with Optimizer for Graph Auto Encoder
2024-02-12 09:42:40,562 [INFO] Creating <class 'torch.optim.adam.Adam'> with lr : 0.01
2024-02-12 09:42:40,565 [INFO] moving model t

# Classification

## Data

In [32]:
associations = get_associations()
y = torch.tensor(associations['increased'].tolist(), dtype=torch.float32).reshape(-1, 1).to(device)

In [33]:
# Train Test Split
train_indices, test_indices = train_test_sampler(y.shape[0], 0.7)

data = SimplePytorchData(md_embed, y)
train_data = SimplePytorchData(md_embed[train_indices], y[train_indices])
test_data = SimplePytorchData(md_embed[test_indices], y[test_indices])

2024-02-12 09:44:07,787 [INFO] Initializing SimplePytorchData with X shape : torch.Size([898, 64]) and y shape : torch.Size([898, 1])
2024-02-12 09:44:07,790 [INFO] Initializing SimplePytorchData with X shape : torch.Size([628, 64]) and y shape : torch.Size([628, 1])
2024-02-12 09:44:07,793 [INFO] Initializing SimplePytorchData with X shape : torch.Size([270, 64]) and y shape : torch.Size([270, 1])


## Classifier

In [34]:
simple_classifier_config = SimpleClassifierConfig()
simple_classifier_config.model_name = "simple classifier"
simple_classifier_config.input_dim = md_embed.shape[1]
simple_classifier_config.hidden_dim = 32
simple_classifier_config.output_dim = 1
simple_classifier_config.num_layers = 2
simple_classifier_config.dropout = 0.1

In [35]:
mda_classifier = SimpleMDAClassifier(simple_classifier_config)

2024-02-12 09:44:07,805 [INFO] Initializing SimpleMDAClassifier with model : simple classifier
2024-02-12 09:44:07,808 [INFO] Initial SimpleMLP with 64 input dimension, 32 hidden dimension, 1 
            output dimension, 2 layers and with 0.1 dropout


## Optimizer

In [36]:
classifier_optimizer_config = OptimizerConfig()
classifier_optimizer_config.optimizer = torch.optim.Adam
classifier_optimizer_config.criterion = torch.nn.BCEWithLogitsLoss()
classifier_optimizer_config.lr = 0.01
classifier_optimizer_config.batch_size = 32
classifier_optimizer_config.n_epoch = 50
classifier_optimizer_config.exp_name = "adam optimizer"
classifier_optimizer_config.save = False
classifier_optimizer_config.save_path = None
classifier_optimizer_config.device = device
classifier_optimizer_config.report_size = 10  # batch to report ratio
classifier_optimizer_config.threshold = 0.5

## Train Test Approach

### Train

In [37]:
train_result = SimpleTrainer().train(model=mda_classifier,
                                     data=train_data,
                                     config=classifier_optimizer_config)

2024-02-12 09:44:07,822 [INFO] Running Simple Trainer with config : adam optimizer
2024-02-12 09:44:07,824 [INFO] moving data and model to cuda
2024-02-12 09:44:07,848 [INFO] loss: 0.0202    [1,    10]
2024-02-12 09:44:07,867 [INFO] loss: 0.0184    [1,    20]
2024-02-12 09:44:07,885 [INFO] loss: 0.0141    [2,    10]
2024-02-12 09:44:07,902 [INFO] loss: 0.0147    [2,    20]
2024-02-12 09:44:07,923 [INFO] loss: 0.0121    [3,    10]
2024-02-12 09:44:07,942 [INFO] loss: 0.0118    [3,    20]
2024-02-12 09:44:07,960 [INFO] loss: 0.0109    [4,    10]
2024-02-12 09:44:07,978 [INFO] loss: 0.0116    [4,    20]
2024-02-12 09:44:07,996 [INFO] loss: 0.0096    [5,    10]
2024-02-12 09:44:08,015 [INFO] loss: 0.0105    [5,    20]
2024-02-12 09:44:08,033 [INFO] loss: 0.0095    [6,    10]
2024-02-12 09:44:08,051 [INFO] loss: 0.0093    [6,    20]
2024-02-12 09:44:08,070 [INFO] loss: 0.0088    [7,    10]
2024-02-12 09:44:08,088 [INFO] loss: 0.0085    [7,    20]
2024-02-12 09:44:08,106 [INFO] loss: 0.0080 

### Test

In [38]:
test_result = SimpleTester().test(model=mda_classifier,
                                  data=test_data,
                                  config=classifier_optimizer_config)

2024-02-12 09:44:09,869 [INFO] Running Simple Tester with config : adam optimizer
2024-02-12 09:44:09,872 [INFO] moving data and model to cuda
2024-02-12 09:44:09,896 [INFO] Result on Test Data : {'AUC': 0.9607385864793678, 'ACC': 0.9407407407407408, 'F1 Score': 0.9407407407407408, 'AUPR': 0, 'Loss': 0.4104647928227981}


In [39]:
test_result.get_result()

{'AUC': 0.9607385864793678,
 'ACC': 0.9407407407407408,
 'F1 Score': 0.9407407407407408,
 'AUPR': 0,
 'Loss': 0.4104647928227981}

## Cross Validation

In [40]:
trainer = SimpleTrainer()
tester = SimpleTester()
factory = SimpleMDAClassifierFactory(simple_classifier_config)
spliter = SimplePytorchDataTrainTestSplit(data)
cross_validation(k=5, data_size=data.X.shape[0], train_test_spliter=spliter, model_factory=factory,
                    trainer=trainer, tester=tester, config=classifier_optimizer_config)

2024-02-12 09:44:09,913 [INFO] Initializing SimpleMDAClassifierFactory with model : simple classifier
2024-02-12 09:44:09,915 [INFO] Initializing SimplePytorchDataTrainTestSplit
2024-02-12 09:44:09,917 [INFO] Start 5-fold Cross Validation with config : adam optimizer
2024-02-12 09:44:09,920 [INFO] ---- Fold 1 ----
2024-02-12 09:44:09,922 [INFO] Initializing SimplePytorchData with X shape : torch.Size([719, 64]) and y shape : torch.Size([719, 1])
2024-02-12 09:44:09,926 [INFO] Initializing SimplePytorchData with X shape : torch.Size([179, 64]) and y shape : torch.Size([179, 1])
2024-02-12 09:44:09,927 [INFO] Initializing SimpleMDAClassifier with model : simple classifier
2024-02-12 09:44:09,928 [INFO] Initial SimpleMLP with 64 input dimension, 32 hidden dimension, 1 
            output dimension, 2 layers and with 0.1 dropout
2024-02-12 09:44:09,930 [INFO] Running Simple Trainer with config : adam optimizer
2024-02-12 09:44:09,931 [INFO] moving data and model to cuda
2024-02-12 09:44:09